In [ ]:
import gym
import gymnasium as gym
from gymnasium import Env, spaces
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.lines as mlines
import matplotlib.patches as patches
import matplotlib.transforms as mtransforms
from matplotlib.legend_handler import HandlerPatch,HandlerTuple
from stable_baselines3 import PPO
from stable_baselines3 import A2C
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback
from plotly.offline import iplot
import plotly.graph_objects as go
import plotly.express as px
import random
import networkx as nx
import matplotlib.patches as mpatches
import torch as th
import itertools






Configurar o Enxame


In [2]:

num_drones = 4
limite_y = 80
limite_x = 100
alcance_comunicacao_jammer = 25
alcance_comunicacao_nos = 5000
B_Hz = 2.4e9
posicoes = np.array([[35, 50], [40, 30], [55, 25],[70,50]])
posicao_jammer = np.array([45, 70])

In [3]:


# Definir a subgrade dentro da área maior
area_size = (100, 100)
subgrade_start = (25, 20)  # Ponto inicial da subgrade (x, y)
subgrade_size = (50, 40)   # Tamanho da subgrade (largura, altura)
resolution = 5             # Resolução da subgrade (distância entre pontos)

# Gerar os pontos discretizados na subgrade
x_points = np.arange(subgrade_start[0], subgrade_start[0] + subgrade_size[0] + 1, resolution)
y_points = np.arange(subgrade_start[1], subgrade_start[1] + subgrade_size[1] + 1, resolution)
grid_points = np.array(np.meshgrid(x_points, y_points)).T.reshape(-1, 2)


VIZUALIZAÇÃO DAS DIREÇÕES DAS ANTENAS ESCOLHIDAS ALEATORIAMENTE

In [ ]:
# Função para calcular a distância entre duas posições
def distancia(pos1, pos2):
    return np.sqrt((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)

# Função para encontrar vizinhos dentro do alcance de comunicação
def encontra_vizinhos(posicoes, alcance_comunicacao_nos):
    num_drones = posicoes.shape[0]
    vizinhos = {}
    for i in range(num_drones):
        vizinhos[i] = []
        for j in range(num_drones):
            if i != j and distancia(posicoes[i], posicoes[j]) <= alcance_comunicacao_nos:
                vizinhos[i].append(j)
    return vizinhos

# Função para verificar quais drones são afetados pelo jammer
def verifica_jammer(posicoes, posicao_jammer, alcance_comunicacao_jammer):
    afetados_pelo_jammer = []
    for i in range(posicoes.shape[0]):
        if distancia(posicoes[i], posicao_jammer) <= alcance_comunicacao_jammer:
            afetados_pelo_jammer.append(i)
    return afetados_pelo_jammer

# Gera direções aleatórias para as antenas
direcoes_antena = np.random.uniform(0, 360, size=num_drones)
# direcoes_antena = [278, 102, 235, 312, 182, 242]
afetados_pelo_jammer=verifica_jammer(posicoes, posicao_jammer, alcance_comunicacao_jammer)
# Comprimento das setas para indicar a direção das antenas
comprimento_seta = 5

# Criar a figura
fig = go.Figure()

# Adiciona os drones
for i in range(num_drones):
    cor = 'blue' if i in afetados_pelo_jammer else 'blue' # type: ignore
    fig.add_trace(go.Scatter(
        x=[posicoes[i, 0]],
        y=[posicoes[i, 1]],
        mode='markers+text',
        marker=dict(size=10, color=cor),
        text=str(i),
        textposition="bottom center",
        showlegend=False
    ))

    # Calcular as coordenadas da seta para a direção da antena
    direcao_radianos = np.radians(direcoes_antena[i])
    dx = comprimento_seta * np.cos(direcao_radianos)
    dy = comprimento_seta * np.sin(direcao_radianos)
    seta_x = posicoes[i, 0] + dx
    seta_y = posicoes[i, 1] + dy
    
    # Adiciona uma linha para indicar a direção da antena
    fig.add_trace(go.Scatter(
        x=[posicoes[i, 0], seta_x],
        y=[posicoes[i, 1], seta_y],
        mode='lines',
        line=dict(color='black', width=1.5),
        showlegend=False
    ))

# Adiciona o jammer
fig.add_trace(go.Scatter(
    x=[posicao_jammer[0]],
    y=[posicao_jammer[1]],
    mode='markers',
    marker=dict(size=12, color='red'),
    name='Jammer'
))

# Adiciona rastros para a legenda
fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers', marker=dict(size=10, color='blue'), name="Drone"))
# fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers', marker=dict(size=10, color='orange'), name="Nó afetado"))

# Configurações do gráfico
fig.update_layout(
    title="Simulação de Modelo de Comunicação Wireless entre Drones",
    xaxis=dict(range=[0, limite_x], autorange=False),
    yaxis=dict(range=[0, limite_y], autorange=False),
    legend_title_text='Legenda',
    legend=dict(traceorder='normal', itemsizing='constant')
)

fig.show()
print(direcoes_antena)

CALCULO DE TODOS OS ANGULOS A USAR

In [ ]:
import numpy as np

# Função para calcular o ângulo entre dois nós em relação à direção da antena
def angulo_entre_nos(pos1, direcao_antena, pos2):
    # Vetor entre os dois nós
    delta_x = pos2[0] - pos1[0]
    delta_y = pos2[1] - pos1[1]
    
    # Direção do vetor em graus
    direcao_vetor_rad = np.arctan2(delta_y, delta_x)
    direcao_vetor_deg = np.degrees(direcao_vetor_rad)
    
    # Diferença entre a direção do vetor e a direção da antena
    angulo = direcao_vetor_deg - direcao_antena
    
    # Ajustar para o intervalo de 0 a 360 graus
    angulo = angulo % 360  # Usa módulo para garantir que o valor esteja entre 0 e 360
    angulo_arredondado = round(angulo)
    
    return angulo_arredondado

# Posições dos drones
# posicoes = np.array([[16, 30], [18, 48], [44, 15], [23, 73], [45, 73], [50, 50]])

# Direções aleatórias das antenas
# direcoes_antena = np.random.uniform(0, 360, size=posicoes.shape[0])

# Matriz para armazenar os ângulos
angulos_matriz = np.zeros((posicoes.shape[0], posicoes.shape[0]))

# Calcular os ângulos para todas as combinações de drones
for i in range(posicoes.shape[0]):
    for j in range(posicoes.shape[0]):
        if i != j:
            angulos_matriz[i, j] = angulo_entre_nos(posicoes[i], direcoes_antena[i], posicoes[j])

# Apresentar os ângulos entre drones sem valores negativos
for i in range(posicoes.shape[0]):
    print(f"Drones que o drone {i} 'vê' com um ângulo:")
    for j in range(posicoes.shape[0]):
        if i != j:
            angulo_arredondado = int(round(angulos_matriz[i, j]))  # Arredondar para unidade
            print(f"  Com o drone {j}: {angulo_arredondado} graus")

PARAMETROS 


In [6]:
Ptx_dBm = 20 
f = 2.4e9 
B_Hz= 2.4e9
d0=1
gamma=2
sigma = 0
c=3e8
lambda_m=c/f
L0=30
potencia_jammer_dBm =100

# Ler o arquivo de ganhos para criar uma tabela de busca
ganhos_df = pd.read_csv('ganhos.csv')

In [ ]:
angulos = np.deg2rad(ganhos_df['angulo'])  # Converter graus para radianos
ganhos = ganhos_df['ganho']

# Criar um gráfico polar
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.plot(angulos, ganhos, linestyle='-', linewidth=1, label='Ganho da Antena')

# Ajustar o layout
ax.set_theta_zero_location('N')  # Configurar a direção Norte como o topo do gráfico
ax.set_theta_direction(-1)  # Configurar a direção dos ângulos para sentido horário
ax.set_rmax(max(ganhos) + 1)  # Ajustar o raio máximo para melhor visualização
ax.set_rticks(np.linspace(min(ganhos), max(ganhos), num=5))  # Definir os ticks para o raio
ax.set_rlabel_position(45)  # Posicionar os labels do raio

# Configurar os ticks para o ângulo
ax.set_xticks(np.deg2rad(np.arange(0, 360, 30)))  # Definir marcas de ângulo a cada 30 graus

# Adicionar legenda e título
ax.legend()
ax.set_title('Diagrama de Radiação', va='bottom')

# Mostrar o gráfico
plt.show()

In [ ]:
def busca_ganhos( angulo):
    # Arredondar para o ângulo inteiro mais próximo
    angulo_ajustado = int(round(angulo))
    
    if angulo_ajustado==360:
        angulo_ajustado=0
    
    # Encontrar o ganho correspondente ao ângulo ajustado
    ganho = ganhos_df.loc[ganhos_df['angulo'] == angulo_ajustado, 'ganho'].iloc[0]
    
    return ganho



# Função para calcular os ganhos de transmissão e recepção
def calcula_ganhos(posicoes, direcoes_antena, indice_transmissor, indice_receptor, ganhos_df):
    # Obter posições e direções
    pos1 = posicoes[indice_transmissor]
    pos2 = posicoes[indice_receptor]
    direcao_antena1 = direcoes_antena[indice_transmissor]
    direcao_antena2 = direcoes_antena[indice_receptor]
    
    # Calcular o ângulo entre os dois nós em relação à direção da antena
    angulo_transmissao = angulo_entre_nos(pos1, direcao_antena1, pos2)
    angulo_rececao = angulo_entre_nos(pos2, direcao_antena2, pos1)
    # print('Angulo receção:',angulo_rececao)
    # print('Angulo tranmissão:',angulo_transmissao)

    # Obter os ganhos para transmissão e recepção
    ganho_transmissao = busca_ganhos( angulo_transmissao)
    ganho_rececao = busca_ganhos( angulo_rececao)
    
    return ganho_transmissao, ganho_rececao

# ganho_transmissao, ganho_rececao = calcula_ganhos(posicoes, direcoes_antena, 0, 1, ganhos_df)



def calcula_potencia_recebida(Ptx_dBm, ganho_transmissao, ganho_rececao, d):
    """
    Calcula a potência do sinal recebido usando a equação de Friis.
    """
    # L = (4 * np.pi * d / lambda_m)**2
    # Prx_dBm = Ptx_dBm + ganho_transmissao + ganho_rececao - 10 * np.log10(L)
    # return Prx_dBm
    # Calcular a perda de percurso
    if d > 0:
        L = L0 + 10 * gamma * np.log10(d / d0) + np.random.normal(0, sigma)
    else:
        L = L0  # Se a distância for zero, assume-se a perda na distância de referência

    # Calcular a potência do sinal recebido
    Prx_dBm = Ptx_dBm + ganho_transmissao + ganho_rececao - L

    return Prx_dBm

def calcula_potencia_jammer(pos_drone, direcao_antena_drone, pos_jammer, potencia_jammer_dBm, ganhos_df):
    # Calcular a distância entre o drone e o jammer
    d = distancia(pos_drone, pos_jammer)
    
    # Calcular o ângulo entre o drone e o jammer
    angulo_entre_drone_e_jammer = angulo_entre_nos(pos_drone, direcao_antena_drone, pos_jammer)
    
    # Obter o ganho do drone em direção ao jammer
    ganho_drone_para_jammer = busca_ganhos( angulo_entre_drone_e_jammer)
    
    # Calcular a potência do ruído recebido usando a equação de Friis
    L = (4 * np.pi * d / lambda_m) ** 2  # Perda de propagação
    potencia_jammer_recebida_dBm = potencia_jammer_dBm + ganho_drone_para_jammer - 10 * np.log10(L)
    
    return potencia_jammer_recebida_dBm

def calcula_capacidade(potencia_sinal_dBm, potencia_ruido_dBm):
    """
    Calcula a capacidade do canal em bits por segundo usando a fórmula de Shannon-Hartley.
    """
    # Converter potência em dBm para Watts
    potencia_sinal_W = 10 ** (potencia_sinal_dBm / 10) / 1000  # Potência do sinal em Watts
    potencia_ruido_W = 10 ** (potencia_ruido_dBm / 10) / 1000  # Potência do ruído em Watts
    
    # Calcular a razão sinal-ruído (SNR)
    snr = potencia_sinal_W / potencia_ruido_W
    
    # Calcular a capacidade do canal em bits por segundo (bps)
    capacidade = B_Hz * np.log2(1 + snr)
    
    return capacidade/1e3

# -------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------------------
for i in range(posicoes.shape[0]):
    for j in range(posicoes.shape[0]):
        if i != j:
            # Calcular a distância entre os dois drones
            d = distancia(posicoes[i], posicoes[j])
            
            # Calcular os ganhos de transmissão e recepção
            ganho_transmissao, ganho_rececao = calcula_ganhos(posicoes, direcoes_antena, i, j, ganhos_df)
            # print(posicoes)
            print('Ganhos Rx,Tx:',ganho_transmissao, ganho_rececao)
            # Calcular a potência do sinal recebido usando a equação de Friis
            potencia_recebida = calcula_potencia_recebida(Ptx_dBm, ganho_transmissao, ganho_rececao, d)
            potencia_ruido = calcula_potencia_jammer(posicoes[j], direcoes_antena[j], posicao_jammer, potencia_jammer_dBm, ganhos_df)
            capacidade_canal=calcula_capacidade(potencia_recebida,potencia_ruido)
            # Imprimir a combinação de nós e a potência do sinal recebido
            print(f"  Potencia recebida do drone {i} para o drone {j}: {potencia_recebida:.2f} dBm")
            print(f"  Ruido recebido no drone {j} e do jammer: {potencia_ruido:.2f} dBm")
            print(f"  Capacidade do canal {i} para {j}: {capacidade_canal:.2f} bps")



In [9]:
seed=0

# Supondo que as seguintes funções são definidas anteriormente no código
# distancia, calcula_ganhos, calcula_potencia_recebida, calcula_potencia_jammer, calcula_capacidade

class UAVCommunicationEnvDjikstra_ppo(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, num_uavs=num_drones, area_size=(100, 90)):
        super().__init__()
        self.num_uavs = num_uavs
        self.area_size = area_size
        self.posicoes = posicoes
        self.posicao_jammer = posicao_jammer
        self.ultimo_info = {}

        # Define o espaço de ação como direções para cada drone
        self.action_space = spaces.MultiDiscrete([360] * num_uavs)
        self.observation_space = spaces.MultiDiscrete([360] * num_uavs)

        self.direcoes_antena = np.random.uniform(0, 360, size=num_uavs)
        self.capacidades = []  # Inicializa a lista de capacidades para normalização
        self.subgrade_size=subgrade_size
        self.subgrade_start=subgrade_start
        self.ganhos_df=ganhos_df

    def step(self, action):
        self.direcoes_antena = np.array(action)
        capacidades_por_link = []
        
        for i in range(self.posicoes.shape[0]):
            for j in range(self.posicoes.shape[0]):
                if i != j:
                    d = distancia(self.posicoes[i], self.posicoes[j])
                    ganho_transmissao, ganho_rececao = calcula_ganhos(self.posicoes, self.direcoes_antena, i, j, ganhos_df)
                    potencia_recebida = calcula_potencia_recebida(Ptx_dBm, ganho_transmissao, ganho_rececao, d, )
                    potencia_ruido = calcula_potencia_jammer(self.posicoes[j], self.direcoes_antena[j], self.posicao_jammer, potencia_jammer_dBm, ganhos_df)
                    capacidade_canal = calcula_capacidade(potencia_recebida, potencia_ruido)
                    capacidades_por_link.append(capacidade_canal)

        capacidade_media = np.mean(capacidades_por_link)
        capacidade_minima = np.min(capacidades_por_link)
        desvio_padrao=np.std(capacidades_por_link)
        # print(desvio_padrao)

        # Armazenar capacidades para normalização
        self.capacidades.extend(capacidades_por_link)
        if len(self.capacidades) > 1000:  # Manter apenas as últimas 1000 capacidades
            self.capacidades = self.capacidades[-1000:]




    # ----------------Normalização MAX-MIN------------------------------------
        # Normalização das capacidades
        min_capacidade = np.min(self.capacidades)
        max_capacidade = np.max(self.capacidades)
        capacidade_media_normalizada = (capacidade_media - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)
        capacidade_minima_normalizada = (capacidade_minima - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)
        
    # ----------------Normalização Z-Score------------------------------------    
        # mean_capacidade = np.mean(capacidades_por_link)
        # std_capacidade = np.std(capacidades_por_link)

        # capacidade_media_normalizada = (capacidade_media - mean_capacidade) / (std_capacidade + 1e-6)
        # capacidade_minima_normalizada = (capacidade_minima - mean_capacidade) / (std_capacidade + 1e-6)

        # Criar um grafo
        G = nx.Graph()
        num_drones = self.posicoes.shape[0]
        capacidade_matriz = np.zeros((self.num_uavs, self.num_uavs))
        link_index = 0

        for i in range(self.num_uavs):
            for j in range(self.num_uavs):
                if i != j:
                    capacidade_matriz[i, j] = capacidades_por_link[link_index]
                    link_index += 1

        # Adicionar arestas ao grafo com o inverso das capacidades como pesos
        for i in range(num_drones):
            for j in range(i + 1, num_drones):
                capacidade = capacidade_matriz[i, j]
                G.add_edge(i, j, weight=1.0 / capacidade)  # Usando o inverso das capacidades como peso

        # Calcular o caminho mais curto usando Dijkstra para cada par de nós
        bottlenecks = {}
        for i in range(num_drones):
            for j in range(num_drones):
                if i != j:
                    try:
                        caminho = nx.dijkstra_path(G, source=i, target=j, weight='weight')
                        # Encontrar o bottleneck no caminho
                        menor_capacidade = float('inf')
                        for k in range(len(caminho) - 1):
                            cap_atual = capacidade_matriz[caminho[k], caminho[k+1]]
                            if cap_atual < menor_capacidade:
                                menor_capacidade = cap_atual
                        bottlenecks[(i, j)] = (caminho, menor_capacidade)
                    except nx.NetworkXNoPath:
                        print(f"Não há caminho do nó {i} para o nó {j}")

        capacidade_bottleneck_media = np.mean([bottleneck for caminho, bottleneck in bottlenecks.values()])
        capacidade_bottleneck_minima = np.min([bottleneck for caminho, bottleneck in bottlenecks.values()])

        # Normalizar capacidades bottleneck
        capacidade_bottleneck_media_normalizada = (capacidade_bottleneck_media - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)
        capacidade_bottleneck_minima_normalizada = (capacidade_bottleneck_minima - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)
        
    #-----------------------------FUNÇÕES OBJETIVO-------------------------------------- 
        
        # recompensa = 0.5 * capacidade_media_normalizada + 0.5 * capacidade_bottleneck_minima_normalizada
        # recompensa= capacidade_media
        # recompensa=capacidade_media/desvio_padrao
        recompensa=capacidade_media*capacidade_bottleneck_minima
        # recompensa=capacidade_media/(capacidade_media-capacidade_bottleneck_minima)
        
        capacidades_por_link = [round(capacidade, 3) for capacidade in capacidades_por_link]
        capacidade_media = round(capacidade_media, 3)
        
        info = {
            'Recompensa': recompensa,
            'Capacidade média [Kbps]': capacidade_media,
            'Capacidades_por_link [Kbps]': capacidades_por_link,
            'Matriz de Capacidades [Kbps]': capacidade_matriz,
            'Capacidade mínima [Kbps]': capacidade_minima,
            'Capacidade média normalizada': capacidade_media_normalizada,
            'Capacidade mínima normalizada': capacidade_minima_normalizada,
            'Capacidade bottleneck média': capacidade_bottleneck_media,
            'Capacidade bottleneck mínima': capacidade_bottleneck_minima,
            'Capacidade bottleneck média normalizada': capacidade_bottleneck_media_normalizada,
            'Capacidade bottleneck mínima normalizada': capacidade_bottleneck_minima_normalizada,
            'Desvio Padrao': desvio_padrao
        }

        done = False
        truncated=False
        return np.array(self.direcoes_antena), recompensa, done,truncated, info

    def render(self, mode='human'):
        sns.set(style="whitegrid")
        fig, ax = plt.subplots(figsize=(8, 6))

        subgrade_start = self.subgrade_start
        subgrade_size = self.subgrade_size
        resolution = 5

        rect = plt.Rectangle(subgrade_start, subgrade_size[0], subgrade_size[1], linewidth=1, edgecolor='blue', facecolor='none')
        ax.add_patch(rect)

        # Desenhar linhas verticais da subgrade dentro do limite da área
        for x in np.arange(subgrade_start[0], subgrade_start[0] + subgrade_size[0] + resolution, resolution):
            ax.axvline(x, color='blue', linestyle='--', linewidth=0.5)

        # Desenhar linhas horizontais da subgrade dentro do limite da área
        for y in np.arange(subgrade_start[1], subgrade_start[1] + subgrade_size[1] + resolution, resolution):
            ax.axhline(y, color='blue', linestyle='--', linewidth=0.5)

        # Desenhar os drones
        for i, pos in enumerate(self.posicoes):
            ax.scatter(pos[0], pos[1], color='blue', s=100, label='Drone' if i == 0 else "")
            ax.text(pos[0], pos[1] - 2, f'{i}', horizontalalignment='center', color='white', fontweight='bold')


        angulos = np.deg2rad(self.ganhos_df['angulo'])
        ganhos = self.ganhos_df['ganho']

        # Normalizar os ganhos para o intervalo [0, 1]
        ganhos_normalizados = (ganhos - ganhos.min()) / (ganhos.max() - ganhos.min())

        # Desenhar o diagrama de radiação sem adicionar ao gráfico principal
        direcao = self.direcoes_antena[0]
        angulos_rotacionados = angulos + np.deg2rad(direcao)
        escala = 6  # Ajuste este valor conforme necessário
        contorno_x = ganhos_normalizados * np.cos(angulos_rotacionados) * escala
        contorno_y = ganhos_normalizados * np.sin(angulos_rotacionados) * escala

        for i, (x, y) in enumerate(self.posicoes):
            direcao = self.direcoes_antena[i]
            angulos_rotacionados = angulos + np.deg2rad(direcao)
            contorno_x = x + ganhos_normalizados * np.cos(angulos_rotacionados) * escala
            contorno_y = y + ganhos_normalizados * np.sin(angulos_rotacionados) * escala
            ax.plot(contorno_x, contorno_y, color='blue', alpha=0.7)

        ax.scatter(self.posicao_jammer[0], self.posicao_jammer[1], color='red', s=100, marker='o', label='Jammer')

        class CustomHandlerPatch(HandlerPatch):
            def create_artists(self, legend, orig_handle, xdescent, ydescent, width, height, fontsize, trans):
                offset_x = width * 0.3  # Ajuste este valor para mover o patch para a direita
                offset_y = ydescent - height * 0.5
                patch = patches.Polygon(
                    np.column_stack((ganhos_normalizados * np.cos(angulos) * escala * 2.5 + offset_x, ganhos_normalizados * np.sin(angulos) * escala * 1.8)), 
                    closed=True, color='blue', alpha=0.9, transform=trans
                )
                return [patch]

        custom_patch = patches.Patch(color='blue', label='Diagrama de Radiação')

        # Adicionar a legenda
        handles = [
            mlines.Line2D([], [], color='blue', marker='o', linestyle='None', markersize=10, label='Drone'),
            mlines.Line2D([], [], color='red', marker='o', linestyle='None', markersize=10, label='Jammer'),
            custom_patch,
            mlines.Line2D([], [], color='none', marker='', linestyle='None', markersize=0, label=' ')  # Espaço em branco
        ]
        
        for i, pos in enumerate(self.posicoes):
            handles.append(
                mlines.Line2D([], [], color='blue', marker='', linestyle='None', markersize=10,
                              label=f'Drone {i}: Pos ({pos[0]}, {pos[1]}), Dir {self.direcoes_antena[i]:.1f}°')
            )
        # ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize='small',borderpad=1, handles=[mlines.Line2D([], [], color='blue', marker='o', linestyle='None', markersize=10, label='Drone'),
        #                                                                                  mlines.Line2D([], [], color='red', marker='o', linestyle='None', markersize=10, label='Jammer'),
        #                                                                                  custom_patch],
        #           handler_map={custom_patch: CustomHandlerPatch()})
        
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize='small',borderpad=1, handles=handles, handler_map={custom_patch: CustomHandlerPatch(), (None, None): HandlerTuple()})
        
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_xlim(0, self.area_size[0])
        ax.set_ylim(0, self.area_size[1])
        ax.set_yticks(np.arange(0, self.area_size[1] + 1, 20))
        plt.title(title)

        plt.show()
        
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.direcoes_antena = np.random.uniform(0, 360, size=self.num_uavs)
        assert len(self.direcoes_antena) == self.num_uavs, "Número incorreto de direções"
        obs = np.array(self.direcoes_antena)
        return obs, {}


COM O DJIKSTRA DENTRO DO AMBIENTE

In [10]:
seed=0

# Supondo que as seguintes funções são definidas anteriormente no código
# distancia, calcula_ganhos, calcula_potencia_recebida, calcula_potencia_jammer, calcula_capacidade

class UAVCommunicationEnvDjikstra(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, num_uavs=num_drones, area_size=(100, 90)):
        super().__init__()
        self.num_uavs = num_uavs
        self.area_size = area_size
        self.posicoes = posicoes
        self.posicao_jammer = posicao_jammer
        self.ultimo_info = {}

        # Define o espaço de ação como direções para cada drone
        self.action_space = spaces.MultiDiscrete([360] * num_uavs)
        self.observation_space = spaces.MultiDiscrete([360] * num_uavs)

        self.direcoes_antena = np.random.uniform(0, 360, size=num_uavs)
        self.capacidades = []  # Inicializa a lista de capacidades para normalização
        self.subgrade_size=subgrade_size
        self.subgrade_start=subgrade_start
        self.ganhos_df=ganhos_df

    def step(self, action):
        self.direcoes_antena = np.array(action)
        capacidades_por_link = []
        
        for i in range(self.posicoes.shape[0]):
            for j in range(self.posicoes.shape[0]):
                if i != j:
                    d = distancia(self.posicoes[i], self.posicoes[j])
                    ganho_transmissao, ganho_rececao = calcula_ganhos(self.posicoes, self.direcoes_antena, i, j, ganhos_df)
                    potencia_recebida = calcula_potencia_recebida(Ptx_dBm, ganho_transmissao, ganho_rececao, d, )
                    potencia_ruido = calcula_potencia_jammer(self.posicoes[j], self.direcoes_antena[j], self.posicao_jammer, potencia_jammer_dBm, ganhos_df)
                    capacidade_canal = calcula_capacidade(potencia_recebida, potencia_ruido)
                    capacidades_por_link.append(capacidade_canal)

        capacidade_media = np.mean(capacidades_por_link)
        capacidade_minima = np.min(capacidades_por_link)
        desvio_padrao=np.std(capacidades_por_link)
        # print(desvio_padrao)

        # Armazenar capacidades para normalização
        self.capacidades.extend(capacidades_por_link)
        if len(self.capacidades) > 1000:  # Manter apenas as últimas 1000 capacidades
            self.capacidades = self.capacidades[-1000:]




    # ----------------Normalização MAX-MIN------------------------------------
        # Normalização das capacidades
        min_capacidade = np.min(self.capacidades)
        max_capacidade = np.max(self.capacidades)
        capacidade_media_normalizada = (capacidade_media - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)
        capacidade_minima_normalizada = (capacidade_minima - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)
        
    # ----------------Normalização Z-Score------------------------------------    
        # mean_capacidade = np.mean(capacidades_por_link)
        # std_capacidade = np.std(capacidades_por_link)

        # capacidade_media_normalizada = (capacidade_media - mean_capacidade) / (std_capacidade + 1e-6)
        # capacidade_minima_normalizada = (capacidade_minima - mean_capacidade) / (std_capacidade + 1e-6)

        # Criar um grafo
        G = nx.Graph()
        num_drones = self.posicoes.shape[0]
        capacidade_matriz = np.zeros((self.num_uavs, self.num_uavs))
        link_index = 0

        for i in range(self.num_uavs):
            for j in range(self.num_uavs):
                if i != j:
                    capacidade_matriz[i, j] = capacidades_por_link[link_index]
                    link_index += 1

        # Adicionar arestas ao grafo com o inverso das capacidades como pesos
        for i in range(num_drones):
            for j in range(i + 1, num_drones):
                capacidade = capacidade_matriz[i, j]
                G.add_edge(i, j, weight=1.0 / capacidade)  # Usando o inverso das capacidades como peso

        # Calcular o caminho mais curto usando Dijkstra para cada par de nós
        bottlenecks = {}
        for i in range(num_drones):
            for j in range(num_drones):
                if i != j:
                    try:
                        caminho = nx.dijkstra_path(G, source=i, target=j, weight='weight')
                        # Encontrar o bottleneck no caminho
                        menor_capacidade = float('inf')
                        for k in range(len(caminho) - 1):
                            cap_atual = capacidade_matriz[caminho[k], caminho[k+1]]
                            if cap_atual < menor_capacidade:
                                menor_capacidade = cap_atual
                        bottlenecks[(i, j)] = (caminho, menor_capacidade)
                    except nx.NetworkXNoPath:
                        print(f"Não há caminho do nó {i} para o nó {j}")

        capacidade_bottleneck_media = np.mean([bottleneck for caminho, bottleneck in bottlenecks.values()])
        capacidade_bottleneck_minima = np.min([bottleneck for caminho, bottleneck in bottlenecks.values()])

        # Normalizar capacidades bottleneck
        capacidade_bottleneck_media_normalizada = (capacidade_bottleneck_media - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)
        capacidade_bottleneck_minima_normalizada = (capacidade_bottleneck_minima - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)
        
    #-----------------------------FUNÇÕES OBJETIVO-------------------------------------- 
        
        # recompensa = 0.5 * capacidade_media_normalizada + 0.5 * capacidade_bottleneck_minima_normalizada
        # recompensa= capacidade_media
        # recompensa=capacidade_media/desvio_padrao
        recompensa=capacidade_media*capacidade_bottleneck_minima
        # recompensa=capacidade_media/(capacidade_media-capacidade_bottleneck_minima)
        
        capacidades_por_link = [round(capacidade, 3) for capacidade in capacidades_por_link]
        capacidade_media = round(capacidade_media, 3)
        
        info = {
            'Recompensa': recompensa,
            'Capacidade média [Kbps]': capacidade_media,
            'Capacidades_por_link [Kbps]': capacidades_por_link,
            'Matriz de Capacidades [Kbps]': capacidade_matriz,
            'Capacidade mínima [Kbps]': capacidade_minima,
            'Capacidade média normalizada': capacidade_media_normalizada,
            'Capacidade mínima normalizada': capacidade_minima_normalizada,
            'Capacidade bottleneck média': capacidade_bottleneck_media,
            'Capacidade bottleneck mínima': capacidade_bottleneck_minima,
            'Capacidade bottleneck média normalizada': capacidade_bottleneck_media_normalizada,
            'Capacidade bottleneck mínima normalizada': capacidade_bottleneck_minima_normalizada,
            'Desvio Padrao': desvio_padrao
        }

        done = False
        # truncated=False
        return np.array(self.direcoes_antena), recompensa, done, info

    def render(self, mode='human'):
        sns.set(style="whitegrid")
        fig, ax = plt.subplots(figsize=(8, 6))

        subgrade_start = self.subgrade_start
        subgrade_size = self.subgrade_size
        resolution = 5

        rect = plt.Rectangle(subgrade_start, subgrade_size[0], subgrade_size[1], linewidth=1, edgecolor='blue', facecolor='none')
        ax.add_patch(rect)

        # Desenhar linhas verticais da subgrade dentro do limite da área
        for x in np.arange(subgrade_start[0], subgrade_start[0] + subgrade_size[0] + resolution, resolution):
            ax.axvline(x, color='blue', linestyle='--', linewidth=0.5)

        # Desenhar linhas horizontais da subgrade dentro do limite da área
        for y in np.arange(subgrade_start[1], subgrade_start[1] + subgrade_size[1] + resolution, resolution):
            ax.axhline(y, color='blue', linestyle='--', linewidth=0.5)

        # Desenhar os drones
        for i, pos in enumerate(self.posicoes):
            ax.scatter(pos[0], pos[1], color='blue', s=100, label='Drone' if i == 0 else "")
            ax.text(pos[0], pos[1] - 2, f'{i}', horizontalalignment='center', color='white', fontweight='bold')


        angulos = np.deg2rad(self.ganhos_df['angulo'])
        ganhos = self.ganhos_df['ganho']

        # Normalizar os ganhos para o intervalo [0, 1]
        ganhos_normalizados = (ganhos - ganhos.min()) / (ganhos.max() - ganhos.min())

        # Desenhar o diagrama de radiação sem adicionar ao gráfico principal
        direcao = self.direcoes_antena[0]
        angulos_rotacionados = angulos + np.deg2rad(direcao)
        escala = 6  # Ajuste este valor conforme necessário
        contorno_x = ganhos_normalizados * np.cos(angulos_rotacionados) * escala
        contorno_y = ganhos_normalizados * np.sin(angulos_rotacionados) * escala

        for i, (x, y) in enumerate(self.posicoes):
            direcao = self.direcoes_antena[i]
            angulos_rotacionados = angulos + np.deg2rad(direcao)
            contorno_x = x + ganhos_normalizados * np.cos(angulos_rotacionados) * escala
            contorno_y = y + ganhos_normalizados * np.sin(angulos_rotacionados) * escala
            ax.plot(contorno_x, contorno_y, color='blue', alpha=0.7)

        ax.scatter(self.posicao_jammer[0], self.posicao_jammer[1], color='red', s=100, marker='o', label='Jammer')

        class CustomHandlerPatch(HandlerPatch):
            def create_artists(self, legend, orig_handle, xdescent, ydescent, width, height, fontsize, trans):
                offset_x = width * 0.3  # Ajuste este valor para mover o patch para a direita
                offset_y = ydescent - height * 0.5
                patch = patches.Polygon(
                    np.column_stack((ganhos_normalizados * np.cos(angulos) * escala * 2.5 + offset_x, ganhos_normalizados * np.sin(angulos) * escala * 1.8)), 
                    closed=True, color='blue', alpha=0.9, transform=trans
                )
                return [patch]

        custom_patch = patches.Patch(color='blue', label='Diagrama de Radiação')

        # Adicionar a legenda
        handles = [
            mlines.Line2D([], [], color='blue', marker='o', linestyle='None', markersize=10, label='Drone'),
            mlines.Line2D([], [], color='red', marker='o', linestyle='None', markersize=10, label='Jammer'),
            custom_patch
        ]
        
        for i, pos in enumerate(self.posicoes):
            handles.append(
                mlines.Line2D([], [], color='blue', marker='', linestyle='None', markersize=10,
                            label=f'Drone {i}: Pos ({pos[0]}, {pos[1]}), Dir {self.direcoes_antena[i]:.1f}°')
            )
        
        handles.append(
            mlines.Line2D([], [], color='none', marker='', linestyle='None', markersize=0,
                        label=f'Capacidade Média [bps]: {cap_media:.2f}')
        )
        handles.append(
            mlines.Line2D([], [], color='none', marker='', linestyle='None', markersize=0,
                        label=f'Bottleneck Mínimo [bps]: {bottleneck:.2f}')
        )
        
        # Ajuste do tamanho da legenda com fontsize e disposição horizontal
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), fontsize='small', ncol=3, handles=handles, handler_map={custom_patch: CustomHandlerPatch()})
        # ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize='small',borderpad=1, handles=handles, handler_map={custom_patch: CustomHandlerPatch(), (None, None): HandlerTuple()})
        
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_xlim(0, self.area_size[0])
        ax.set_ylim(0, self.area_size[1])
        ax.set_yticks(np.arange(0, self.area_size[1] + 1, 20))
    
        plt.title(title)

        plt.show()

        
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.direcoes_antena = np.random.uniform(0, 360, size=self.num_uavs)
        assert len(self.direcoes_antena) == self.num_uavs, "Número incorreto de direções"
        obs = np.array(self.direcoes_antena)
        return obs, {}


CONTROLO


In [ ]:
# Criar uma instância do ambiente

env = UAVCommunicationEnvDjikstra()
# Definir direções e posições escolhidas manualmente
direcoes_escolhidas = np.array([210, 366, 164, 310], dtype=np.float32)
posicoes_escolhidas = np.array([[35, 50], [40, 30], [55, 25], [70, 50]])

# Atualizar o ambiente com as direções e posições escolhidas
env.direcoes_antena = direcoes_escolhidas
env.posicoes = posicoes_escolhidas

# Renderizar o estado do ambiente
title='Controlo'

cap_media=0
bottleneck=0
env.render()



# Se quiser obter a recompensa e outros detalhes
state, reward, terminated, info = env.step(env.action_space.sample())
print('Direções Estabelecidas:\n', state)
print("\nRecompensa:\n", reward)
if 'Capacidade média [Kbps]' in info:
    print("\nCapacidade Média [Kbps]:", info['Capacidade média [Kbps]'])
if 'Capacidade média [Kbps]' in info:
    print("\nCapacidade de Bottleneck Minima [Kbps]:", info['Capacidade bottleneck mínima'])
if 'Capacidades_por_link [Kbps]' in info:
    print("\nCapacidade por Link [Kbps]:", info['Capacidades_por_link [Kbps]'])
if 'Matriz de Capacidades [Kbps]' in info:
    # capacidade_matriz=info['Matriz de Capacidades [Kbps]']
    print("\nMatriz de Capacidades [Kbps]:\n", '\n'.join(['\t' + ' '.join(f"{item:.2f}" for item in row) for row in info['Matriz de Capacidades [Kbps]']]))
else:
    print("Chave 'Matriz de Capacidades [Kbps]' não encontrada em info.")

ALGORITMO GENETICO

1.Tamanho da População:

    Maior: Uma população maior oferece uma diversidade genética mais ampla, o que pode ajudar a evitar mínimos locais, mas também aumenta o custo computacional.
    Menor: Uma população menor pode levar a uma convergência mais rápida, mas pode ficar presa em mínimos locais.

2.Taxa de Mutação:

    Alta: Uma taxa de mutação mais alta aumenta a diversidade genética dentro da população e pode ajudar a sair de mínimos locais. No entanto, se for muito alta, pode causar uma busca aleatória, reduzindo a eficiência do algoritmo.
    Baixa: Uma taxa de mutação baixa mantém a estabilidade da população, mas pode resultar em convergência prematura para soluções subótimas.

3.Taxa de Cruzamento:

    Alta: Promove a combinação de características de diferentes indivíduos, potencialmente levando a uma melhoria mais rápida.
    Baixa: Reduz a probabilidade de alterar boas soluções, mas pode limitar a capacidade do algoritmo de explorar novas e potencialmente melhores áreas do espaço de busca.

4.Número de Gerações:

    Maior número de gerações: Permite mais iterações do processo de seleção, cruzamento e mutação, dando mais tempo para a população evoluir.
    Menor número de gerações: Pode ser suficiente se a convergência for rápida, mas corre o risco de não permitir desenvolvimento suficiente, especialmente com uma taxa de mutação baixa.

In [12]:
class GeneticAlgorithm:
    def __init__(self, env, population_size=50, generations=50, mutation_rate=0.15, crossover_rate=0.9):
        self.env = env  # O ambiente do UAV
        self.population_size = population_size  # Tamanho da população
        self.generations = generations  # Número de gerações
        self.mutation_rate = mutation_rate  # Taxa de mutação
        self.crossover_rate = crossover_rate  # Taxa de cruzamento
        
        # Criação de população inicial com valores aleatórios
        self.population = [self.random_chromosome() for _ in range(population_size)]
        
        # Verificar tamanho da população
        assert len(self.population) == self.population_size, "Tamanho da população incorreto"
        for chrom in self.population:
            assert len(chrom) == self.env.num_uavs, "Número de genes no cromossomo está errado"

    # Cria um cromossomo aleatório (direções das antenas para cada drone)
    def random_chromosome(self):
        return np.random.uniform(0, 360, size=self.env.num_uavs)

    # Avalia a aptidão de um cromossomo
    def evaluate_fitness(self, chromosome):
        obs, _ = self.env.reset()
        action = np.array(chromosome)
        try:
            obs, reward, done, info = self.env.step(action)
            return (reward, info)  # Certifique-se de retornar uma tupla
        except Exception as e:
            print(f"Erro: {e}")
            return (0, {})

    # Seleção por torneio
    def select(self):
        tournament_size = 5
        selected = []
        fitness_scores = [self.evaluate_fitness(chrom) for chrom in self.population]
        for _ in range(self.population_size):
            contenders = random.sample(list(enumerate(fitness_scores)), tournament_size)
            # Acessa o valor de fitness para comparação
            winner_index = max(contenders, key=lambda item: item[1][0])[0] 
            selected.append(self.population[winner_index])
        return selected

    # Cruzamento (crossover) entre dois pais
    def crossover(self, parent1, parent2):
        if random.random() < self.crossover_rate:
            # Ponto de cruzamento
            point = random.randint(1, self.env.num_uavs - 1)
            # Mistura os genes dos pais para criar filhos
            child1 = np.concatenate([parent1[:point], parent2[point:]])
            child2 = np.concatenate([parent2[:point], parent1[point:]])
            return child1, child2
        else:
            return parent1, parent2

    # Aplica mutação a um cromossomo
    def mutate(self, chromosome):
        for i in range(len(chromosome)):
            if random.random() < self.mutation_rate:
                mutation = random.uniform(-50, 50)
                chromosome[i] = (chromosome[i] + mutation) % 360
        return chromosome

    # Executa o algoritmo genético
    def run(self):
        best_info = None
        best_fitness_values = []  # Armazena os valores de Best Fitness ao longo das gerações

        for generation in range(self.generations):
            new_population = []
            parents = self.select()  # Seleciona os melhores pais
            
            while len(new_population) < self.population_size:
                parent1, parent2 = random.sample(parents, 2)
                child1, child2 = self.crossover(parent1, parent2)
                new_population.extend([self.mutate(child1), self.mutate(child2)])  # Adiciona filhos
            
            self.population = new_population[:self.population_size]  # Atualiza a população
            # Calcula a aptidão de todos os cromossomos na população
            fitness_scores = [self.evaluate_fitness(chrom) for chrom in self.population]
            best_fitness, best_info = max(fitness_scores, key=lambda x: x[0])
            
            best_fitness_values.append(best_fitness)  # Salva o Best Fitness para esta geração
            
            print(f"Generation {generation}: Best Fitness = {best_fitness:.2f}")
        
        # Adiciona os valores de Best Fitness ao best_info antes de retornar
        if best_info is not None:
            best_info['Best Fitness Values'] = best_fitness_values
        
        # Retorna a melhor solução encontrada junto com os valores de Best Fitness
        best_solution_index = max(range(len(self.population)), key=lambda i: fitness_scores[i][0])
        best_solution = self.population[best_solution_index]
        
        return best_solution, best_info
    
    def reset(self):
        self.direcoes_antena = np.random.uniform(0, 360, size=self.num_uavs)
        assert len(self.direcoes_antena) == self.num_uavs, "Direções das antenas estão incorretas"
        obs = np.array(self.direcoes_antena)
        return obs, {}

ALGORITMO GENETICO

In [ ]:
env = UAVCommunicationEnvDjikstra()

# Criação do algoritmo genético
ga = GeneticAlgorithm(env)

# Executar o algoritmo genético
best_solution_genetico, best_info_genetico = ga.run()

# Renderizar a melhor solução
env.direcoes_antena = np.array([210, 366, 164, 310], dtype=np.float32)

title='Direções Estabelecidas - Algoritmo Genético'
bottleneck=best_info_genetico['Capacidade bottleneck mínima']
cap_media=best_info_genetico['Capacidade média [Kbps]']
env.render()

print("Direções das Antenas:", best_solution_genetico)

if 'Capacidade média [Kbps]' in best_info_genetico:
    print("\nValor de Recompensa:", best_info_genetico['Recompensa'])
    print("\nCapacidade Média:", best_info_genetico['Capacidade média [Kbps]'])
if 'Capacidade média [Kbps]' in best_info_genetico:
    print("\nCapacidade Bottleneck Minima:", best_info_genetico['Capacidade bottleneck mínima'])
if 'Capacidades_por_link [Kbps]' in best_info_genetico:
    print("\nCapacidade por Link:", best_info_genetico['Capacidades_por_link [Kbps]'])
if 'Matriz de Capacidades [Kbps]' in best_info_genetico:
    # capacidade_matriz=info['Matriz de Capacidades [Kbps]']
    print("\nMatriz de Capacidades:\n", '\n'.join(['\t' + ' '.join(f"{item:.2f}" for item in row) for row in info['Matriz de Capacidades [Kbps]']]))
else:
    print("Chave 'Matriz de Capacidades [Kbps]' não encontrada em info.")

# Plotando os valores de Best Fitness
if 'Best Fitness Values' in best_info_genetico:
    best_fitness_values = best_info_genetico['Best Fitness Values']
    plt.figure(figsize=(10, 6))
    plt.plot(best_fitness_values, marker='o', linestyle='-', color='b', label='Best Fitness')
    plt.title('Evolução dos Valores da FO durante o GA')
    plt.xlabel('Generation')
    plt.ylabel('Best Fitness')
    plt.grid(True)
    plt.legend()

    plt.show()
else:
    print("Chave 'Best Fitness Values' não encontrada em best_info_genetico.")

In [ ]:
# Plotando os valores de Best Fitness
if 'Best Fitness Values' in best_info_genetico:
    best_fitness_values = best_info_genetico['Best Fitness Values']
    generations = range(len(best_fitness_values))
    plt.figure(figsize=(10, 6))
    plt.plot(best_fitness_values, marker='o', linestyle='-', color='b', label='Best Fitness-Directional')
    plt.plot(generations, [0.11] * len(generations), marker='o', linestyle='-', color='orange', label='Best Fitness - Omnidirectional')
    
    plt.title('')
    plt.xlabel('Generation')
    plt.ylabel('')
    plt.grid(True)
    plt.legend()

    plt.show()
else:
    print("Chave 'Best Fitness Values' não encontrada em best_info_genetico.")

ALGORITMO PPO

In [ ]:
class RewardCallback(BaseCallback):
    def __init__(self, check_freq: int, verbose: int = 1):
        super(RewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.rewards = []
        self.value_losses = []  # Para armazenar os value losses

    def _on_step(self) -> bool:
        # A cada check_freq passos, calcula a recompensa média e salva
        if self.n_calls % self.check_freq == 0:
            # Calcula a recompensa média nos últimos check_freq timesteps
            mean_reward = np.mean(self.locals['rewards'][-self.check_freq:])
            self.rewards.append(mean_reward)

            # Armazena o value loss atual
            if 'value_loss' in self.locals:
                self.value_losses.append(self.locals['value_loss'])

        return True

# Função de treinamento PPO com callback
def train_ppo_with_callback(env, total_timesteps=10000, learning_rate=3e-1, n_steps=2048, batch_size=64, n_epochs=20, gamma=0.99, gae_lambda=0.95, clip_range=0.2, check_freq=50):
    # Criar uma instância do ambiente vetorizado
    env = DummyVecEnv([lambda: env])

    # Criar o modelo PPO com parâmetros adicionais
    model = PPO("MlpPolicy", env, verbose=1,
                learning_rate=learning_rate,
                n_steps=n_steps,
                batch_size=batch_size,
                n_epochs=n_epochs,
                gamma=gamma,
                gae_lambda=gae_lambda,
                clip_range=clip_range)

    # Criar o callback para salvar as recompensas e value losses
    reward_callback = RewardCallback(check_freq=check_freq)

    # Treinar o modelo
    model.learn(total_timesteps=total_timesteps, callback=reward_callback)
    model.save("ppo_uav_model")

    return model, reward_callback

# Função para plotar recompensas
def plot_rewards(rewards, check_freq):
    plt.figure(figsize=(10, 6))
    plt.plot(np.arange(len(rewards)) * check_freq, rewards, linestyle='-', color='b', label='Average Reward')
    plt.title('')
    plt.xlabel('Timesteps')
    plt.ylabel('')
    plt.grid(True)
    plt.legend()
    plt.show()

# Criação do ambiente
env = UAVCommunicationEnvDjikstra_ppo()

# Treinamento com PPO usando o callback para salvar as recompensas médias
print('-----------------Treinar o Modelo-----------------------------')
model_ppo, reward_callback = train_ppo_with_callback(env)

# Plotar a evolução das recompensas médias
plot_rewards(reward_callback.rewards, check_freq=50)

# Exibir o gráfico
plt.show()


In [ ]:

# Carregar o modelo salvo
model_ppo = PPO.load("ppo_uav_model")

env = UAVCommunicationEnvDjikstra()
# Obter a melhor solução do PPO
obs = env.reset()
done = False
best_state_ppo = None
best_reward_ppo = -np.inf
best_info_ppo = {}

print('-----------------Aplicar o Modelo-----------------------------')

num_cycles=1000
for cycle in range(num_cycles):
    # print(cycle)
    obs, _ = env.reset()
    done = False
    total_reward = 0
    
    action, _ = model_ppo.predict(obs)
    obs, reward, done, info = env.step(action)
    total_reward += reward

    if total_reward > best_reward_ppo:
        best_reward_ppo = total_reward
        best_state_ppo = obs
        best_info_ppo = info

# Renderizar a melhor solução do PPO
env.direcoes_antena = best_state_ppo

title='Direções Estabelecidas - PPO'
bottleneck=best_info_ppo['Capacidade bottleneck mínima']
cap_media=best_info_ppo['Capacidade média [Kbps]']
env.render()

print("Direções das Antenas - PPO:", best_state_ppo)
if 'Capacidade média [Kbps]' in best_info_ppo:
    print("\nRecompensa:", best_info_ppo['Recompensa'])
    print("\nCapacidade Média:", best_info_ppo['Capacidade média [Kbps]'])
if 'Capacidade média [Kbps]' in best_info_ppo:
    print("\nCapacidade Bottleneck Minima:", best_info_ppo['Capacidade bottleneck mínima'])
if 'Capacidades_por_link [Kbps]' in best_info_ppo:
    print("\nCapacidade por Link:", best_info_ppo['Capacidades_por_link [Kbps]'])
if 'Matriz de Capacidades [Kbps]' in best_info_ppo:
    print("\nMatriz de Capacidades:\n", '\n'.join(['\t' + ' '.join(f"{item:.2f}" for item in row) for row in best_info_ppo['Matriz de Capacidades [Kbps]']]))
else:
    print("Chave 'Matriz de Capacidades [Kbps]' não encontrada em best_info.")

BUSCA EXAUSTIVA

In [ ]:
# Função para calcular a recompensa de acordo com a descrição fornecida



def calcula_recompensa(direcoes_antena, capacidades_totais):
    capacidades_por_link = []
    
    for i in range(posicoes.shape[0]):
        for j in range(posicoes.shape[0]):
            if i != j:
                d = distancia(posicoes[i], posicoes[j])
                ganho_transmissao, ganho_rececao = calcula_ganhos(posicoes, direcoes_antena, i, j, ganhos_df)
                potencia_recebida = calcula_potencia_recebida(Ptx_dBm, ganho_transmissao, ganho_rececao, d)
                potencia_ruido = calcula_potencia_jammer(posicoes[j], direcoes_antena[j], posicao_jammer, potencia_jammer_dBm, ganhos_df)
                capacidade_canal = calcula_capacidade(potencia_recebida, potencia_ruido)
                capacidades_por_link.append(capacidade_canal)

    capacidade_media_exaustiva = np.mean(capacidades_por_link)
    desvio_padrao_exaustiva=np.std(capacidades_por_link)
    
    capacidade_minima = np.min(capacidades_por_link)

    capacidades_totais.extend(capacidades_por_link)
    if len(capacidades_totais) > 1000:
        capacidades_totais = capacidades_totais[-1000:]


    # print('Capac. por link:\n',capacidades_por_link)
    
    min_capacidade = np.min(capacidades_totais)
    max_capacidade = np.max(capacidades_totais)
    # capacidade_media_normalizada = (capacidade_media - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)
    # capacidade_minima_normalizada = (capacidade_minima - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)

    G = nx.Graph()
    num_drones = posicoes.shape[0]
    capacidade_matriz = np.zeros((num_drones, num_drones))
    link_index = 0

    for i in range(num_drones):
        for j in range(num_drones):
            if i != j:
                capacidade_matriz[i, j] = capacidades_por_link[link_index]
                link_index += 1

    for i in range(num_drones):
        for j in range(i + 1, num_drones):
            capacidade = capacidade_matriz[i, j]
            G.add_edge(i, j, weight=1.0 / capacidade)

    bottlenecks = {}
    for i in range(num_drones):
        for j in range(num_drones):
            if i != j:
                try:
                    caminho = nx.dijkstra_path(G, source=i, target=j, weight='weight')
                    menor_capacidade = float('inf')
                    for k in range(len(caminho) - 1):
                        cap_atual = capacidade_matriz[caminho[k], caminho[k+1]]
                        if cap_atual < menor_capacidade:
                            menor_capacidade = cap_atual
                    bottlenecks[(i, j)] = (caminho, menor_capacidade)
                except nx.NetworkXNoPath:
                    print(f"Não há caminho do nó {i} para o nó {j}")

    capacidade_bottleneck_media = np.mean([bottleneck for caminho, bottleneck in bottlenecks.values()])
    capacidade_bottleneck_minima = np.min([bottleneck for caminho, bottleneck in bottlenecks.values()])

    capacidade_bottleneck_media_normalizada = (capacidade_bottleneck_media - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)
    capacidade_bottleneck_minima_normalizada = (capacidade_bottleneck_minima - min_capacidade) / (max_capacidade - min_capacidade + 1e-6)

    # recompensa = 0.5 * capacidade_media_normalizada + 0.5 * capacidade_bottleneck_minima_normalizada
    # recompensa=capacidade_media_exaustiva
    # recompensa=capacidade_media_exaustiva/desvio_padrao_exaustiva
    recompensa=capacidade_media_exaustiva*capacidade_bottleneck_minima
    # recompensa=capacidade_media_exaustiva/(capacidade_media_exaustiva-capacidade_bottleneck_minima)
    
    
    # recompensa = 0.001 * capacidade_media + 0.009 * capacidade_bottleneck_minima

    return recompensa, capacidade_media_exaustiva, capacidade_bottleneck_media,capacidade_bottleneck_minima, desvio_padrao_exaustiva

# Realizar a busca exaustiva
melhor_recompensa = -float('inf')
melhores_direcoes = None
capacidades_totais = []

for direcoes in itertools.product(range(0, 360, 2), repeat=num_drones):
    direcoes = np.array(direcoes)
    recompensa, capacidade_media, capacidade_bottleneck_media,capacidade_bottleneck_minima,desvio_padrao_exaustiva = calcula_recompensa(direcoes, capacidades_totais)
    
    if recompensa > melhor_recompensa:
        melhor_recompensa = recompensa
        melhor_capacidade_media = capacidade_media
        melhor_capacidade_bottleneck = capacidade_bottleneck_media
        # capacidade_media_normalizada_final=capacidade_media_normalizada
        # capacidade_bottleneck_media_normalizada_final=capacidade_bottleneck_media_normalizada
        capacidade_bottleneck_minima_final=capacidade_bottleneck_minima
        desvio_padrao_exaustiva_final=desvio_padrao_exaustiva
        melhores_direcoes = np.array(direcoes)
        
        
env=UAVCommunicationEnvDjikstra()
env.direcoes_antena=melhores_direcoes
env.render()
# Exibir os resultados
print("Melhores Direções das Antenas (Busca Exaustiva):", melhores_direcoes)
print("Melhor Capacidade Média [Kbps] (Busca Exaustiva):", melhor_capacidade_media)
print("Melhor Capacidade Bottleneck Média [Kbps] (Busca Exaustiva):", melhor_capacidade_bottleneck)
print("Melhor Recompensa (Busca Exaustiva):", melhor_recompensa)

In [ ]:

print("Resultados do Algoritmo Genético:")
print("Direções das Antenas:", best_solution)
print("Capacidade Média:", best_info['Capacidade média [Kbps]'])
print("Capacidade Bottleneck Mínima:", best_info['Capacidade bottleneck mínima'])
# print("Capacidade Média Normalizada:", best_info['Capacidade média normalizada'])
print("Capacidade Bottleneck Média :", best_info['Capacidade bottleneck média'])
print("Desvio Padrão :", best_info['Desvio Padrao'])
# print("Capacidade Bottleneck Média Normalizada:", best_info['Capacidade bottleneck média normalizada'])
print("\nRecompensa:", best_info['Recompensa'])


print("\nResultados da Busca Exaustiva:")
print("Melhores Direções das Antenas:", melhores_direcoes)
print("Melhor Capacidade Média:", melhor_capacidade_media)
print("Melhor Capacidade Bottleneck Mínima:", capacidade_bottleneck_minima_final)
# print("Capacidade Média Normalizada:", capacidade_media_normalizada_final)
print("Melhor Capacidade Bottleneck Média:", melhor_capacidade_bottleneck)
print("Desvio padrão:",desvio_padrao_exaustiva_final)
# print("Melhor Capacidade Bottleneck Média Normalizada:", capacidade_bottleneck_media_normalizada_final)
print("\nMelhor Recompensa:", melhor_recompensa)

env.direcoes_antena=best_solution_genetico
env.render()



direcoes_antena_exaustiva = melhores_direcoes



# Atualizar o ambiente com as direções e posições escolhidas
env.direcoes_antena = melhores_direcoes


# Renderizar o estado do ambiente
title='Busca Exaustiva'
env.render()